In [1]:
import sys


sys.path.append('../')

In [2]:
from adafruit_dotstar_pi_animation.data import *
from adafruit_dotstar_pi_animation.player import SimplePixelBufPlayer

import adafruit_dotstar
import board


num_leds = 11
pixels = adafruit_dotstar.DotStar(board.SCLK, board.MOSI, num_leds, pixel_order=adafruit_dotstar.GBR, auto_write=False)

In [92]:
import dataclasses
import random


def generate_animation(len_of_single_led_cycle_ms, step_len_ms, num_leds, animation_total_len_ms, chance_pixel_stops_cycle):
    red = Pixel(red=255, green=0, blue=0, brightness=1.0)
    green = Pixel(red=0, green=255, blue=0, brightness=1.0)
    white = Pixel(red=255, green=255, blue=255, brightness=1.0)
    
    current_time = 0
    frames = []
    brightness_step_size = 1 / (len_of_single_led_cycle_ms / step_len_ms)
    pixel_choices = [red, green, white]
    direction_to_starting_brightness_mapping = {
        -1: 1.0,
        1: 0.1
    }
    direction_choices = list(direction_to_starting_brightness_mapping.keys())
    starting_brightness_choices = list(direction_to_starting_brightness_mapping.values())
    

    # Seed this thing with a random assortment of red/green choices, random brightness, and a random indicator for if 
    # we're getting brighter or dimmer

    
    pixel_brightness_directions = [random.choice(direction_choices) for _ in range(num_leds)]
    starting_pixels = []
    
    for b in pixel_brightness_directions:
        # We make the brightness random here so that the pixels aren't all on the same phase at the same time
        new_pixel = dataclasses.replace(
            random.choice(pixel_choices), 
            brightness=random.random())
        starting_pixels.append(new_pixel)
    
    last_frame = Frame(pixels=starting_pixels, display_ms=step_len_ms)
    while current_time < animation_total_len_ms:
        # Frame loop
        
        next_frames_pixels = []
        for p_idx, p in enumerate(last_frame.pixels):
            # Pixel loop
            # Step 1: Did this pixel randomly die?
            # if random.random() > chance_pixel_stops_cycle:
            #     # Yep, it's dead. Generate a totally new pixel
            #     pixel_brightness_directions[p_idx] = random.choice(direction_choices)
            #     next_frames_pixels.append(
            #         dataclasses.replace(random.choice(pixel_choices), 
            #                             brightness=direction_to_starting_brightness_mapping[pixel_brightness_directions[p_idx]]))
            #     continue
            
            new_brightness = p.brightness + (pixel_brightness_directions[p_idx] * brightness_step_size)
            
            # Step 2: Is it at max/min brightness?
            if new_brightness >= 1.0 or new_brightness <= 0.1:
                # Yep, it's done. Generate a totally new pixel
                pixel_brightness_directions[p_idx] = random.choice(direction_choices)
                next_frames_pixels.append(
                    dataclasses.replace(random.choice(pixel_choices), 
                                        brightness=random.random()))
                continue
                
            # Step 3: None of the above? Advance it's brightness progression
            next_frames_pixels.append(
                dataclasses.replace(p, brightness=new_brightness))
        
        current_time = current_time + step_len_ms
        frames.append(Frame(pixels=next_frames_pixels, display_ms=step_len_ms))
        last_frame = frames[-1]
        
    return Animation(frames=frames, max_plays=1, loop_infinitely=False, pause_between_play_ms=0)

In [104]:
test_animation = generate_animation(750, 5, num_leds, 10000, 1.0)
test_animation.loop_infinitely = True
player = SimplePixelBufPlayer(pixels)
player.load(test_animation)
player.play()

KeyboardInterrupt: 

In [48]:
for f in test_animation.frames:
    for p in f.pixels:
        if p.brightness != 0.0 and p.brightness != 1.0:
            print(p.brightness)

In [103]:
pixels.fill((0, 0, 0))
pixels.show()